# DataFrames in Spark (Part 1)

This course has been designed with Spark 2.0.1 in mind (Oct 2016).


# 1. Overview

## 1.1. RDDs versus DataFrames

What is Spark SQL?
- Spark SQL takes basic RDDs and puts a schema on them.

What are schemas?
- Schema = Table Names + Column Names + Column Types

What are the pros of schemas?
- Schemas enable using column names instead of column positions
- Schemas enable queries using SQL and DataFrame syntax
- Schemas make your data more structured.

What is a DataFrame?
- DataFrames are the primary abstraction in Spark SQL.
- Think of a DataFrames as RDDs with schema.

What is a schema?
- Schemas are metadata about your data.
- Schemas define table names, column names, and column types over your data.
- Schemas enable using SQL and DataFrame syntax to query your RDDs, instead of using column positions.

# 2. Operational DataFrames in Python

We'll proceed along the usual spark flow (see above).
1. create the enviromnent to run Spark SQL from python
2. create DataFrames from RDDs or from files
3. run some transformations
4. execute actions to obtain values (local objects in python)

## 2.1. Initializing a `SparkContext` and `SqlContext` in Python

Using:

```python
import pyspark as ps
sc = ps.SparkContext('local[4]')
```

will create a *"local"* cluster made of the driver using all 4 cores.


In [ ]:
import pyspark as ps    # for the pyspark suite
import warnings         # for displaying warning

In [ ]:
try:
    # we try to create a SparkContext to work locally on all cpus available
    sc = ps.SparkContext('local[4]')
    print("Just created a SparkContext")
except ValueError:
    # give a warning if SparkContext already exists (for use inside pyspark)
    warnings.warn("SparkContext already exists in this scope")

Then we create a `SQLContext` using our `SparkContext` as argument.

In [ ]:
sqlContext = ps.SQLContext(sc)

## 2.2. Creating a DataFrame

### 2.2.1. From an RDD (specifying schema)

You can create a DataFrame from an existing RDD (whatever source you used to create this one), if you add a schema.

To build a schema, you will use existing data types provided in the [`pyspqrk.sql.types`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.types) module. Here's a list of the most useful ones (subjective criteria).

| Types | Python-like type |
| - | - |
| StringType | string |
| IntegerType | int |
| FloatType | float |
| ArrayType\* | array or list |
| MapType | dict |

\* see later UDF functions on how to use that

In [ ]:
# remember that csv file ?
def casting_function((id, date, store, state, product, amount)):
    return((int(id), date, int(store), state, int(product), float(amount)))

rdd_sales = sc.textFile('data/sales.csv')\
        .map(lambda rowstr : rowstr.split(","))\
        .filter(lambda row: not row[0].startswith('#'))\
        .map(casting_function)

rdd_sales.collect()

In [ ]:
# import the many data types
from pyspark.sql.types import *

# create a schema of your own
schema = StructType( [
    StructField('id',IntegerType(),True),
    StructField('date',StringType(),True),
    StructField('store',IntegerType(),True),
    StructField('state',StringType(),True),
    StructField('product',IntegerType(),True),
    StructField('amount',FloatType(),True) ] )

# feed that into a DataFrame
df = sqlContext.createDataFrame(rdd_sales,schema)

# show the result
df.show()

# print the schema
df.printSchema()

### 2.2.2. Reading from files (infering schema)

Use [`sqlContext.read.csv`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.csv) to load a CSV into a DataFrame. You can specify every useful parameter in there. It can infer the schema.

In [ ]:
# read CSV
df = sqlContext.read.csv('data/sales.csv',
                         header=True,       # use headers or not
                         quote='"',         # char for quotes
                         sep=",",           # char for separation
                         inferSchema=True)  # do we infer schema or not ?

# prints the schema
df.printSchema()

# some functions are still valid
print("line count: {}".format(df.count()))

# show the table in a oh-so-nice format
df.show()

Use [`sqlContext.read.json`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.json) to load a JSON file into a DataFrame. You can specify every useful parameter in there. It can infer the schema.

In [ ]:
# read JSON
df = sqlContext.read.json('data/sales.json')

# prints the schema
df.printSchema()

# some functions are still valid
print("line count: {}".format(df.count()))

# show the table in a oh-so-nice format
df.show()

In [ ]:
# read JSON
df = sqlContext.read.json('data/sales2.json.gz')

# show the table in a oh-so-nice format
df.show()

## 2.3. Actions : turning your DataFrame into a local object

Some actions just remain the same, you won't have to learn Spark all over again.

Some new actions give you the possibility to describe and show the content in a more fashionable manner.

When used/executed in IPython or in a notebook, they **launch the processing of the DAG**. This is where Spark stops being **lazy**. This is where your script will take time to execute.

| Method | DF vs RDD? | Description |
| - | - | - |
| [`.collect()`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.collect) | identical | Return a list that contains all of the elements as Rows. |
| [`.count()`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.count) | identical | Return the number of elements. |
| [`.take(n)`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.take) | identical | Take the first `n` elements. |
| [`.top(n)`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.top) | identical | Get the top `n` elements. |
| [`.first()`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.first) | identical | Return the first element. |
| [`.show(n)`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.show) | <span style="color:green">new</span> | Show the DataFrame in table format (`n=20` by default) |
| [`.toPandas()`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.toPandas) | <span style="color:green">new</span> | Convert the DF into a Pandas DF. |
| [`.printSchema(*cols)`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.printSchema)\* | <span style="color:green">new</span> | Display the schema. This is not an action, it doesn't launch the DAG, but it fits better in this category. |
| [`.describe(*cols)`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.describe) | <span style="color:green">new</span> | Compute statistics for this column. |
| [`.sum(*cols)`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.sum) | <span style="color:red">different</span> | Applies on GroupedData only (see transformations). |
| [`.mean(*cols)`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.mean) | <span style="color:red">different</span> | Applies on GroupedData only (see transformations). |
| [`.min(*cols)`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.min) | <span style="color:red">different</span> | Applies on GroupedData only (see transformations). |
| [`.max(*cols)`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.max) | <span style="color:red">different</span> | Applies on GroupedData only (see transformations). |


In [ ]:
# read CSV
df_sales = sqlContext.read.csv('data/sales.csv',
                         header=True,       # use headers or not
                         quote='"',         # char for quotes
                         sep=",",           # char for separation
                         inferSchema=True)  # do we infer schema or not ?

In [ ]:
df_sales.show()

In [ ]:
df_sales.toPandas()

This is how `.collect()` returns things...

In [ ]:
df_sales.collect()

In [ ]:
# prints the schema
print("--- printSchema()")
df_sales.printSchema()

# prints the table itself
print("--- show()")
df_sales.show()

# show the statistics of all numerical columns
print("--- describe()")
df_sales.describe().show()

# show the statistics of one specific column
print("--- describe(Amount)")
df_sales.describe("Amount").show()

## 2.3. Transformations on DataFrames

- They are still **lazy**: Spark doesn't apply the transformation right away, it just builds on the **DAG**
- They transform a DataFrame into another because DataFrames are also **immutable**.
- They can be **wide** or **narrow** (whether they shuffle partitions or not).

You got that... DataFrames are just RDDs with a schema.

| Method | Type | Category | Description |
| - | - | - |
| [`.map(func)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map) | transformation | mapping | Return a new RDD by applying a function to each element of this RDD. |
| [`.flatMap(func)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.flatMap) | transformation | mapping | Return a new RDD by first applying a function to all elements of this RDD, and then flattening the results. |
| [`.filter(func)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.filter) | transformation | reduction |  Return a new RDD containing only the elements that satisfy a predicate. |
| [`.sample()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.sample) | transformation | reduction | Return a sampled subset of this RDD. |
| [`.distinct()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.distinct) | transformation | reduction |  Return a new RDD containing the distinct elements in this RDD. |
| [`.keys()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.keys) | transformation | `<k,v>` | Return an RDD with the keys of each tuple. |
| [`.values()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.values) | transformation | `<k,v>` | Return an RDD with the values of each tuple. |
| [`.join(rddB)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.join) | transformation | `<k,v>` | Return an RDD containing all pairs of elements with matching keys in self and other. Each pair of elements will be returned as a (k, (v1, v2)) tuple, where (k, v1) is in self and (k, v2) is in other. |
| [`.reduceByKey()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) | transformation | `<k,v>` | Merge the values for each key using an associative and commutative reduce function. |
| [`.groupByKey()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey) | transformation | `<k,v>` | Merge the values for each key using non-associative operation, like mean. |
| [`.sortBy(keyfunc)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.sortBy) | transformation | sorting |  Sorts this RDD by the given keyfunc. |
| [`.sortByKey()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.sortByKey) | transformation | sorting/`<k,v>` | Sorts this RDD, which is assumed to consist of (key, value) pairs. |



### 2.3.2. `.withColumn()`: adding column using operations or functions

#### `.withColumn("label", func)` : 

In [ ]:
# read CSV
df_aapl = sqlContext.read.csv('data/aapl.csv',
                         header=True,       # use headers or not
                         quote='"',         # char for quotes
                         sep=",",           # char for separation
                         inferSchema=True)  # do we infer schema or not ?

df_aapl.show(5)

df_aapl.printSchema()

#### `.withColumn("label", func)` : constant value

In [ ]:
from pyspark.sql.functions import lit

df_out = df_aapl.withColumn("blabla", lit("echo"))

df_out.show(5)

#### `.withColumn("label", func)` : column operations

In [ ]:
df_out = df_aapl.withColumn("diff", df_aapl.High - df_aapl.Low)

df_out.show(5)

#### `.withColumn("label", func)` : user defined function

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

def my_specialfunc(h,l,o,c):
    return ((h-l)*(o-c))

my_specialfunc_udf = udf(lambda h,l,o,c : my_specialfunc(h,l,o,c), DoubleType())

df_out = df_aapl.withColumn("special", my_specialfunc_udf(df_aapl.High, df_aapl.Low, df_aapl.Open, df_aapl.Close))

df_out.show()

#### `.select(*cols)` : selecting specific columns

In [ ]:
df_out = df_aapl.select(["Open", "Close"])

df_out.show(5)

#### `.groupBy()`: aggregating in DataFrames

In [ ]:
from pyspark.sql import functions as F

df_out = df_sales.groupBy("State").agg(F.sum("Amount"))

df_out.show()

#### `.orderBy()` : sorting by a column

In [ ]:
df_out = df_sales.groupBy("State").agg(F.sum("Amount")).orderBy("sum(Amount)", ascending=False)

df_out.show()

# 3. Let's design chains of transformations together ! (reloaded)

## 3.1. Computing sales per state

### Input DataFrame

In [ ]:
# read CSV
df_sales = sqlContext.read.csv('data/sales.csv',
                         header=True,       # use headers or not
                         quote='"',         # char for quotes
                         sep=",",           # char for separation
                         inferSchema=True)  # do we infer schema or not ?

df_sales.show()

### Task

You want to obtain a sorted ~~RDD~~ DataFrame of the states in which you have most sales done (amount).

What transformations do you need to apply ?
If you had to draw a workflow of the transformations to apply ?

### Code

In [ ]:
df_out = df_sales

df_out.show()

## Solution (use your mouse to uncover)

<span style="color:white;font-family:'Courier New'"><br/>
df_out = df_sales.groupBy(df_sales.State)\<br/>
                 .agg(F.sum(df_sales.Amount).alias('Money'))\<br/>
                 .orderBy("Money", ascending=False)<br/>
<br/>
df_out.show()<br/>
</span>

## 3.2. Find the date on which AAPL's stock price was the highest

### Input DataFrame

In [ ]:
# read CSV
df_aapl = sqlContext.read.csv('data/aapl.csv',
                         header=True,       # use headers or not
                         quote='"',         # char for quotes
                         sep=",",           # char for separation
                         inferSchema=True)  # do we infer schema or not ?

df_aapl.show(5)

### Task

Now, design a pipeline that would :

1. ~~filter out headers and last line~~
2. ~~split each line based on comma~~
3. keep only fields for Date ~~(col 0)~~ and Close ~~(col 4)~~
4. order by Close in descending order

### Code

In [ ]:
df_out = df_aapl # apply transformation here...

df_out.show(5)

### Solution

<span style="color:white;font-family:'Courier New'">
df_out.select("Close", "Date").orderBy(df_aapl.Close, ascending=False).show(5)<br/>
</span>


# 4. Machine Learning on DataFrames

http://spark.apache.org/docs/latest/ml-features.html

In [ ]:
# read CSV
df_aapl = sqlContext.read.csv('data/aapl.csv',
                         header=True,       # use headers or not
                         quote='"',         # char for quotes
                         sep=",",           # char for separation
                         inferSchema=True)  # do we infer schema or not ?

df_aapl.show(5)

df_aapl.printSchema()

In [ ]:
from pyspark.ml.feature import MinMaxScaler, VectorAssembler

# assemble values in a vector
vectorAssembler = VectorAssembler(inputCols=["Close"],
                                  outputCol="features")

df_vector = vectorAssembler.transform(df_aapl)

scaler = MinMaxScaler(inputCol="features", outputCol="scaledfeatures")

# Compute summary statistics and generate MinMaxScalerModel
scalerModel = scaler.fit(df_vector)

# rescale each feature to range [min, max].
scaledData = scalerModel.transform(df_vector)
scaledData.select("features", "scaledfeatures").show(5)